# Local Linear Iterative Stockholder Analysis (L-ISA) schemes

First, we need to import the required packages. In this tutorial, we will use `LinearISAWPart`, which includes various L-ISA schemes.

In [ ]:
import logging

import numpy as np
from setup import prepare_argument_dict, prepare_grid_and_dens, print_results

from horton_part import BasisFuncHelper, LinearISAWPart

logging.basicConfig(level=logging.INFO, format="%(levelname)s:    %(message)s")

Next, we will continue to use a water molecule as an example and prepare the molecular grid and density using the helper function `prepare_grid_and_dens`. The input for this function is the filename of a Gaussian checkpoint file with an `fchk` extension.

In [ ]:
mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")

There are two ways to address the optimization problem of partitioning a molecule into atomic contributions:

- Directly solving the constrained non-linear optimization problem.
- Solving non-linear equations.

These two methods are equivalent in principle.

## Non-linear Optimization Problem

In this section, we focus on the first approach: directly solving the non-linear optimization problem. The local constrained optimization can be solved in various ways, either by direct minimization with the constraints taken into account, or by solving the resulting local non-linear Euler-Lagrange equations. The latter is equivalent to solving non-linear equations or fixed-point problems.

In the `horton-part` package, two solvers are employed for direct minimization:

- Convex optimization (101)
- Trust-region method (301, 302)

Meanwhile, three solvers are used for solving non-linear equations:

- Self-consistent method (201)
- Newton method (202)
- Direct inversion in iterative space (203)

Different numbers are used to identify each corresponding solver.

### Convex Optimization Method



In [ ]:
def convex_optimization():
    """Local LISA by solving convex optimization problem."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 101
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


convex_optimization()

### Trust-Region Method

TODO: The trust-region method, which operates implicitly, is attributed to the third category. However, it can be demonstrated that it is equivalent to the first category, and thus we introduce it here.

In [ ]:
def trust_region_implicitly():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 301
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


trust_region_implicitly()

One can also explicitly add constraints. This method falls into the first category, that is, direct minimization.

In [ ]:
def trust_region_explicitly():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 302
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


trust_region_explicitly()

## Non-linear Equations (Fixed-point Equations)

Direct minimization can be transformed into a set of non-linear equations using non-linear Euler-Lagrange equations, which are also known as fixed-point equations. Therefore, they can be solved by different methods, and we have implemented three distinct solvers.

### Self-Consistent Method

The first one is called the self-consistent method, an iterative approach to solving fixed-point equations by using the previous solution as the input for the next step. It has been proven to always converge in a finite number of iterations to any given convergence criteria, making it a robust method. Another feature of this method is that it always guarantees non-negative parameters during the minimization.

However, one technical issue with this method is that slow convergence can arise in practice.

In [ ]:
def self_consistent_method():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 201
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


self_consistent_method()

### Direct Inversion in Iterative Space (DIIS)

This method has been extensively used to solve self-consistent field (SCF) problems in the fields of quantum chemistry and physics. In this tutorial, we employ this method to accelerate the solving of fixed-point problems.

It should be noted that one potential issue with this method is that non-negative parameters cannot be guaranteed during optimization in the conventional DIIS approach. Although this issue can be addressed by explicitly introducing constraints to the linear combination coefficients, key concepts in DIIS, numerical issues may still arise, such as singular matrices or convergence problems.

In [ ]:
def diis_method():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 202
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


diis_method()

### Newton Method

The final method is the Newton method. Similarly to the previous methods, the Newton method cannot guarantee non-negative parameters, and negative pro-atom densities may arise during optimization. To address this, one might need to modify the hyper-parameters used in the method for different systems, which can make it less robust.

In [ ]:
def newton_method():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 203
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


newton_method()

## Customized Methods

One can also apply customized solvers. Next, we will implement and apply a modified version of the self-consistent method.

In [ ]:
def customize_self_consistent_solver(
    bs_funcs, rho, propars, points, weights, threshold, density_cutoff
):
    r"""
    Optimize parameters for proatom density functions using a self-consistent (SC) method.

    .. math::

        N_{Ai} = \int \rho_A(r) \frac{\rho_{Ai}^0(r)}{\rho_A^0(r)} dr

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    density_cutoff : float
        Density values below this cutoff are considered invalid.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.

    """
    pro_shells = bs_funcs * propars[:, None]
    pro_density = np.einsum("ij->j", pro_shells)
    sick = (rho < density_cutoff) | (pro_density < density_cutoff)
    ratio = np.divide(rho, pro_density, out=np.zeros_like(rho), where=~sick)
    propars[:] = np.einsum("p,ip->i", weights, pro_shells * ratio)
    return propars

In [ ]:
def customized_self_consistent_method():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = customize_self_consistent_solver
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


customized_self_consistent_method()

One can also apply customized basis functions using the `BasisFuncHelper` class. First, we need to define the basis functions, which should adhere to the following general format:

$$
  f_{ak} = c_{ak} \exp^{-\alpha_{ak} |r|^{n_{ak}}}
$$

Here, $c_{ak}$ represents the prefactors and the pro-atom parameters to be determined; $\alpha_{ak}$ is the exponential coefficient; and $n_{ak}$ corresponds to the power of the exponential function. The subscripts $a$ and $k$ denote the indices of atoms and basis functions, respectively.

We initialize `BasisFuncHelper` using the default method, and therefore need to provide the $\alpha_{ak}$ (`exponents`), $n_{ak}$ (`exponents_orders`), and initial values of $c_{ak}$ (`initials`). To illustrate the functionality, we use three exponential functions with different $n$ values, i.e., 1 (Slater type), 1.5, and 2 (Gaussian type).

In [ ]:
bs_dict = {
    "exponents_orders": {1: [1, 1.5, 2], 8: [1, 1.5, 2]},
    "exponents": {1: [0.1, 0.2, 0.3], 8: [1.0, 2.0, 3.0]},
    "initials": {1: [0.33] * 3, 8: [0.33] * 3},
}
basis_helper = BasisFuncHelper(**bs_dict)

It should be noted that one can also initialize a `BasisFuncHelper` through a `json` file by calling `BasisFuncHelper.from_json('json_file_path.json')`.

Next, we can apply the newly customized basis functions and the self-consistent solver. It is important to note

In [ ]:
def customized_basis_functions():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = customize_self_consistent_solver
    kwargs["basis_func"] = basis_helper
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


customized_basis_functions()